In [1]:
import time
import numpy as np
from Conways import*

pygame 2.0.2 (SDL 2.0.16, Python 3.9.4)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
env = conways(size = 10, max_it=1000)
print(env.run())

36


In [4]:
x = np.zeros(shape=(10, 10), dtype=bool)
x[1, 1] = True
x[1, 2] = True
x[2, 1] = True
x[2, 2] = True
env.reset(x, offset=False)
actions = env.run()
print(actions)

inf


In [17]:
alpha = 0.1 
max_iterations = 100000
gamma = 0.995

total = 0
tot_reward = 0
prev_reward = 0

q_table_row_limit = 10000

epsilon = 1
epsilon_decay = 0.99995

In [18]:
q_table = np.random.uniform(low=0, high=1, size=[q_table_row_limit, 10, 10])
q_table.shape

print(np.argmax(q_table[0]))
print(np.max(q_table[0]))
print(q_table[0])

75
0.9909090589988636
[[0.9544469  0.0264927  0.58260812 0.53226602 0.08865391 0.2529748
  0.48987564 0.09578756 0.47623496 0.45176145]
 [0.50046517 0.6477062  0.9337155  0.83052136 0.15565624 0.43012585
  0.17954245 0.83462772 0.57125971 0.08252308]
 [0.26629205 0.00217541 0.4115352  0.36756171 0.3423428  0.81144546
  0.72682225 0.97082154 0.66485546 0.60361303]
 [0.77367114 0.65359095 0.38091909 0.74214715 0.7499489  0.86570896
  0.46848551 0.72276159 0.53648038 0.87349239]
 [0.25253733 0.71168414 0.32609835 0.22706621 0.63918735 0.06601085
  0.37619379 0.37231    0.34229078 0.6275968 ]
 [0.83208441 0.63807721 0.42711394 0.7263233  0.26947078 0.96994862
  0.20384892 0.49106344 0.90350827 0.23901594]
 [0.47509496 0.96205216 0.23265837 0.70427175 0.44420872 0.6280781
  0.29786189 0.76967381 0.6313666  0.6216556 ]
 [0.15649299 0.53409749 0.51805897 0.79090234 0.2690596  0.99090906
  0.9821608  0.20480494 0.45334941 0.06392757]
 [0.77558411 0.5621822  0.12048103 0.68021309 0.83323832 0.7

In [21]:

state_score = np.array([float('-inf') for i in range(q_table_row_limit)])
state_table = [0 for i in range(q_table_row_limit)]

def get_discrete_state(s):
    for i in range(q_table_row_limit):
        if state_table[i] and np.array_equal(s, state_table[i]):
            return i
    i = np.argmin(state_score)
    state_table[i] = np.copy(s)
    reinitialize q_table entry
    return i


In [70]:
current_state = np.zeros(shape=(10, 10), dtype=bool)

#take initial action
discrete_state = get_discrete_state(current_state)
index = np.argmax(q_table[discrete_state])
action = (index/10, index%10)
current_state[action[0], action[1]] = True
best_state = current_state
env.reset(current_state)
state_score[discrete_state] = env.run()
best_reward = np.copy(current_state)

for i in range(max_iterations):
    t0 = time.time()
    

    if (i % 2000 == 0):
        print(f"Episode: {i}")

    reward = 0
    #test possible actions after taking that action
    for j in range(q_table_row_limit-1):
        if np.random.random() > epsilon:
            index = np.argmax(q_table[discrete_state])
            action = (index/10, index%10)
        else:
            action = (np.random.randint(0, 10), np.random.randint(0, 10))

        current_state[action[0], action[1]] = True
        env.reset(current_state)

        new_discrete_state = get_discrete_state(current_state)

        state_score[discrete_state] = env.run()
        reward += state_score[discrete_state]

        if (state_score[discrete_state] > best_reward):
            best_state = np.copy(current_state)
            best_reward = state_score[discrete_state]

        max_future_q = np.max(q_table[new_discrete_state])
        current_q = q_table[(discrete_state, action[0], action[1])]

        new_q = (1 - alpha) * current_q + alpha * (state_score[discrete_state] + gamma * max_future_q)
        q_table[(discrete_state, action[0], action[1])] = new_q

        discrete_state = new_discrete_state
    
    
    current_state = np.copy(best_state)
    reward = np.max(state_score)
    if epsilon > 0.05: 
        if reward > prev_reward and i > 10000:
            epsilon = epsilon_decay ** (i - 10000)

            if i % 500 == 0:
                print("Epsilon: " + str(epsilon))
    
    t1 = time.time()
    iteration_time = t1-t0
    total += iteration_time

    tot_reward += reward
    prev_reward = reward

    if i % 1000 == 0: 
        mean = total / 1000
        print("Time Average: " + str(mean))
        total = 0

        mean_reward = tot_reward / 1000
        print("Mean Reward: " + str(mean_reward))
        tot_reward = 0

Episode: 0


NameError: name 'discrete_state' is not defined

Final = best_state